Dataset Exploration

In [1]:
import pandas as pd

df = pd.read_csv('Lead Scoring.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

Feature Selection step

In [2]:
selected_columns= ['Lead Source','Do Not Email', 'Do Not Call', 'Country', 'Country','Specialization', 'What is your current occupation', 'Through Recommendations' ,'City', 'I agree to pay the amount through cheque', 'Last Notable Activity','Converted']

selected_df= df[selected_columns]

print(selected_df.head())



      Lead Source Do Not Email Do Not Call Country Country  \
0      Olark Chat           No          No     NaN     NaN   
1  Organic Search           No          No   India   India   
2  Direct Traffic           No          No   India   India   
3  Direct Traffic           No          No   India   India   
4          Google           No          No   India   India   

            Specialization What is your current occupation  \
0                   Select                      Unemployed   
1                   Select                      Unemployed   
2  Business Administration                         Student   
3    Media and Advertising                      Unemployed   
4                   Select                      Unemployed   

  Through Recommendations    City I agree to pay the amount through cheque  \
0                      No  Select                                       No   
1                      No  Select                                       No   
2                   

Handling Missing Values

In [3]:
selected_df.isnull().sum()


Lead Source                                   36
Do Not Email                                   0
Do Not Call                                    0
Country                                     2461
Country                                     2461
Specialization                              1438
What is your current occupation             2690
Through Recommendations                        0
City                                        1420
I agree to pay the amount through cheque       0
Last Notable Activity                          0
Converted                                      0
dtype: int64

A way to handle missing country and city values is to deduce them from each other, here The variable city_country_map now contains a dictionary of City → Country pairs, which allows quick lookup of which country a given city belongs to.

after running the below code, the missing values of countries dropped from 2461 to 838


In [4]:
# Extract unique City-Country pairs from non-missing data
city_country_map = df.dropna(subset=['City', 'Country']).set_index('City')['Country'].to_dict()

print(city_country_map)
# Use the mapping to fill missing Country based on City
df['Country'] = df.apply(
    lambda row: city_country_map.get(row['City'], row['Country']),
    axis=1
)

{'Select': 'India', 'Mumbai': 'India', 'Thane & Outskirts': 'India', 'Other Metro Cities': 'India', 'Other Cities': 'Bangladesh', 'Other Cities of Maharashtra': 'India', 'Tier II Cities': 'India'}


In [5]:
print(city_country_map)

{'Select': 'India', 'Mumbai': 'India', 'Thane & Outskirts': 'India', 'Other Metro Cities': 'India', 'Other Cities': 'Bangladesh', 'Other Cities of Maharashtra': 'India', 'Tier II Cities': 'India'}


fll Missing Cities Using Country (Optional)
For rows where Country is known but City is missing:Fills with most frequent city in a country

In [6]:
# Create a Country-City mapping (most frequent city per country)
country_city_map = df.dropna(subset=['City', 'Country']).groupby('Country')['City'].agg(pd.Series.mode).to_dict()
 # Ensure it has key-value pairs like {'Cairo': 'Egypt', 'Tokyo': 'Japan'}
# Fill missing City based on Country
df['City'] = df.apply(
    lambda row: country_city_map.get(row['Country'], row['City']),
    axis=1
)

In [7]:
selected_df.isnull().sum()


Lead Source                                   36
Do Not Email                                   0
Do Not Call                                    0
Country                                     2461
Country                                     2461
Specialization                              1438
What is your current occupation             2690
Through Recommendations                        0
City                                        1420
I agree to pay the amount through cheque       0
Last Notable Activity                          0
Converted                                      0
dtype: int64

For Specialization, occupation, and lead source we will replace missing values with the mode:

In [8]:
df['Specialization'].value_counts(normalize=True) * 100 # check if there is a single dominant value


Specialization
Select                               24.891054
Finance Management                   12.509613
Human Resource Management            10.869008
Marketing Management                 10.740836
Operations Management                 6.447065
Business Administration               5.165342
IT Projects Management                4.691105
Supply Chain Management               4.473212
Banking, Investment And Insurance     4.332223
Media and Advertising                 2.601897
Travel and Tourism                    2.601897
International Business                2.281466
Healthcare Management                 2.037939
Hospitality Management                1.461164
E-COMMERCE                            1.435529
Retail Management                     1.281723
Rural and Agribusiness                0.935658
E-Business                            0.730582
Services Excellence                   0.512689
Name: proportion, dtype: float64

replacing missing specialization values with'select'
1. Preserves User Behavior
"Select" directly reflects that the user did not choose a specialization from the dropdown/list.

This maintains the integrity of the data collection process (e.g., a blank or unselected field maps to "Select").

2. Avoids Bias
Using the mode (most frequent specialization) would artificially inflate the importance of that category.

Example: If "Marketing" is the mode, imputing it for 24% of missing values would mislead the model into thinking Marketing is more prevalent than it truly is.

3. Retains Predictive Signal
Leads who left Specialization as "Select" might behave differently (e.g., lower conversion rates).

The model can learn from this pattern if "Select" is treated as a distinct category.

In [9]:
df['Specialization'].fillna('Select', inplace=True)


C:\Users\islam\AppData\Local\Temp\ipykernel_22264\3944417538.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Specialization'].fillna('Select', inplace=True)


In [10]:
df['Specialization'].value_counts(normalize=True) * 100 # notic that the percentage of the select value increased now


Specialization
Select                               36.580087
Finance Management                   10.562771
Human Resource Management             9.177489
Marketing Management                  9.069264
Operations Management                 5.443723
Business Administration               4.361472
IT Projects Management                3.961039
Supply Chain Management               3.777056
Banking, Investment And Insurance     3.658009
Media and Advertising                 2.196970
Travel and Tourism                    2.196970
International Business                1.926407
Healthcare Management                 1.720779
Hospitality Management                1.233766
E-COMMERCE                            1.212121
Retail Management                     1.082251
Rural and Agribusiness                0.790043
E-Business                            0.616883
Services Excellence                   0.432900
Name: proportion, dtype: float64

Let's do the same anlysis for the occupation column

In [11]:
df['What is your current occupation'].value_counts(normalize=True) * 100



What is your current occupation
Unemployed              85.496183
Working Professional    10.778626
Student                  3.206107
Other                    0.244275
Housewife                0.152672
Businessman              0.122137
Name: proportion, dtype: float64

Upon examination, we might omit the occupation column as it might lead to noise. the values in it are irrelevant to our B2B audience


In [12]:
df.drop(columns=['What is your current occupation', 'Prospect ID', 'Lead Number', 'Lead Origin', 'TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit', 'Last Activity', 'How did you hear about X Education', 'What matters most to you in choosing a course', 'Search', 'Magazine', 'Newspaper Article', 'X Education Forums', 'Newspaper', 'Digital Advertisement', 'Receive More Updates About Our Courses', 'Lead Quality', 'Update me on Supply Chain Content', 'Get updates on DM Content', 'Lead Profile', 'Asymmetrique Activity Index', 'Asymmetrique Profile Index', 'Asymmetrique Activity Score', 'Asymmetrique Profile Score', 'A free copy of Mastering The Interview','I agree to pay the amount through cheque' ], inplace=True)

In [13]:
df.head()

,Lead Source,Do Not Email,Do Not Call,Converted,Country,Specialization,Through Recommendations,Tags,City,Last Notable Activity
0,Olark Chat,No,No,0,India,Select,No,Interested in other courses,Mumbai,Modified
1,Organic Search,No,No,0,India,Select,No,Ringing,Mumbai,Email Opened
2,Direct Traffic,No,No,1,India,Business Administration,No,Will revert after reading the email,Mumbai,Email Opened
3,Direct Traffic,No,No,0,India,Media and Advertising,No,Ringing,Mumbai,Modified
4,Google,No,No,1,India,Select,No,Will revert after reading the email,Mumbai,Modified


In [14]:
df['Lead Source']

0           Olark Chat
1       Organic Search
2       Direct Traffic
3       Direct Traffic
4               Google
             ...      
9235    Direct Traffic
9236    Direct Traffic
9237    Direct Traffic
9238            Google
9239    Direct Traffic
Name: Lead Source, Length: 9240, dtype: object

In [15]:
import numpy as np

# Define possible authority levels
authority_levels = [5, 4, 3, 2, 1]  # From highest to lowest

# Randomly assign decision authority
df['Decision_Authority'] = np.random.choice(authority_levels, size=len(df))

# Save the updated dataset
df.to_csv('updated_leads.csv', index=False)

print(df[['Decision_Authority']].head())


   Decision_Authority
0                   5
1                   5
2                   1
3                   2
4                   3


In [16]:
df['Decision_Authority'] = df['Lead Source'].map({
    'Referral': 5,  # Referrals might be direct senior-level contacts
    'LinkedIn': 4,  # LinkedIn tends to have mid-to-senior professionals
    'Website': 3,   # Website inquiries can be managers or specialists
    'Ad Campaign': 2,  # Cold leads often include analysts and entry-level roles
    'Other': 1      # Catch-all for low-authority leads
}).fillna(np.random.choice(authority_levels))  # Assign random if no match


In [17]:
df['Decision_Authority']

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
9235    1.0
9236    1.0
9237    1.0
9238    1.0
9239    1.0
Name: Decision_Authority, Length: 9240, dtype: float64

In [18]:
# Define job positions corresponding to authority levels
job_titles_map = {
    5: ['CEO', 'CFO', 'CTO', 'COO', 'Chief Innovation Officer'],
    4: ['Vice President', 'Director of Operations', 'Head of Marketing', 'Senior Strategy Director'],
    3: ['Project Manager', 'Marketing Manager', 'Team Lead', 'Operations Manager'],
    2: ['Data Analyst', 'Software Engineer', 'Business Consultant', 'Technical Specialist'],
    1: ['Intern', 'Junior Developer', 'Assistant Coordinator', 'Associate Analyst']
}


# Generate 'Job Position' column by assigning random titles from the dictionary
df['Job Position'] = df['Decision_Authority'].apply(lambda x: np.random.choice(job_titles_map.get(x, ['Unknown'])))

# Save the updated dataset
df.to_csv('updated_leads.csv', index=False)

print(df[['Decision_Authority', 'Job Position']].head())

   Decision_Authority           Job Position
0                 1.0                 Intern
1                 1.0  Assistant Coordinator
2                 1.0                 Intern
3                 1.0      Associate Analyst
4                 1.0       Junior Developer


In [19]:
df

,Lead Source,Do Not Email,Do Not Call,Converted,Country,Specialization,Through Recommendations,Tags,City,Last Notable Activity,Decision_Authority,Job Position
0,Olark Chat,No,No,0,India,Select,No,Interested in other courses,Mumbai,Modified,1.0,Intern
1,Organic Search,No,No,0,India,Select,No,Ringing,Mumbai,Email Opened,1.0,Assistant Coordinator
2,Direct Traffic,No,No,1,India,Business Administration,No,Will revert after reading the email,Mumbai,Email Opened,1.0,Intern
3,Direct Traffic,No,No,0,India,Media and Advertising,No,Ringing,Mumbai,Modified,1.0,Associate Analyst
4,Google,No,No,1,India,Select,No,Will revert after reading the email,Mumbai,Modified,1.0,Junior Developer
...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Direct Traffic,Yes,No,1,India,IT Projects Management,No,Will revert after reading the email,Mumbai,Email Marked Spam,1.0,Associate Analyst
9236,Direct Traffic,No,No,0,India,Media and Advertising,No,wrong number given,Mumbai,SMS Sent,1.0,Intern
9237,Direct Traffic,Yes,No,0,India,Business Administration,No,invalid number,Mumbai,SMS Sent,1.0,Assistant Coordinator
9238,Google,No,No,1,India,Human Resource Management,No,NaN,Mumbai,SMS Sent,1.0,Associate Analyst


In [20]:
df['Decision_Authority'].value_counts()

Decision_Authority
1.0    9240
Name: count, dtype: int64

The proposed method of using the lead Source turned out not to be a good approach because of the dominance of the Decision Authority value of 2. We researched another method, which is to randomize the values based on probability distribution of professional hierarchy.

In [21]:
import numpy as np

# Define authority levels and their probabilities (adjusted for balance)
authority_levels = [5, 4, 3, 2, 1]
probabilities = [0.05, 0.15, 0.25, 0.35, 0.20]  # More realistic distribution

# Assign Decision_Authority using weighted probability
df['Decision_Authority'] = np.random.choice(authority_levels, size=len(df), p=probabilities)


In [22]:
job_titles_map = {
    5: ['CEO', 'CFO', 'CTO', 'COO', 'Chief Innovation Officer'],
    4: ['Vice President', 'Director of Operations', 'Head of Marketing', 'Senior Strategy Director'],
    3: ['Project Manager', 'Marketing Manager', 'Team Lead', 'Operations Manager'],
    2: ['Data Analyst', 'Software Engineer', 'Business Consultant', 'Technical Specialist'],
    1: ['Intern', 'Junior Developer', 'Assistant Coordinator', 'Associate Analyst']
}


# Generate 'Job Position' column by assigning random titles from the dictionary
df['Job Position'] = df['Decision_Authority'].apply(lambda x: np.random.choice(job_titles_map.get(x, ['Unknown'])))

# Save the updated dataset
df.to_csv('updated_leads.csv', index=False)

print(df[['Decision_Authority', 'Job Position']].head())

   Decision_Authority           Job Position
0                   1      Associate Analyst
1                   4      Head of Marketing
2                   1  Assistant Coordinator
3                   2           Data Analyst
4                   3     Operations Manager


Insights: The probabilistic approach exhibits good variations that will help our model generalize.
 Now it's time to finish the data preprocessing steps:

In [23]:
df

,Lead Source,Do Not Email,Do Not Call,Converted,Country,Specialization,Through Recommendations,Tags,City,Last Notable Activity,Decision_Authority,Job Position
0,Olark Chat,No,No,0,India,Select,No,Interested in other courses,Mumbai,Modified,1,Associate Analyst
1,Organic Search,No,No,0,India,Select,No,Ringing,Mumbai,Email Opened,4,Head of Marketing
2,Direct Traffic,No,No,1,India,Business Administration,No,Will revert after reading the email,Mumbai,Email Opened,1,Assistant Coordinator
3,Direct Traffic,No,No,0,India,Media and Advertising,No,Ringing,Mumbai,Modified,2,Data Analyst
4,Google,No,No,1,India,Select,No,Will revert after reading the email,Mumbai,Modified,3,Operations Manager
...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Direct Traffic,Yes,No,1,India,IT Projects Management,No,Will revert after reading the email,Mumbai,Email Marked Spam,5,CFO
9236,Direct Traffic,No,No,0,India,Media and Advertising,No,wrong number given,Mumbai,SMS Sent,5,CTO
9237,Direct Traffic,Yes,No,0,India,Business Administration,No,invalid number,Mumbai,SMS Sent,3,Team Lead
9238,Google,No,No,1,India,Human Resource Management,No,NaN,Mumbai,SMS Sent,2,Business Consultant


Handling Missing values

In [24]:
df.isnull().sum()

Lead Source                  36
Do Not Email                  0
Do Not Call                   0
Converted                     0
Country                     838
Specialization                0
Through Recommendations       0
Tags                       3353
City                        865
Last Notable Activity         0
Decision_Authority            0
Job Position                  0
dtype: int64

In [25]:
df['Tags']

0               Interested in other courses
1                                   Ringing
2       Will revert after reading the email
3                                   Ringing
4       Will revert after reading the email
                       ...                 
9235    Will revert after reading the email
9236                     wrong number given
9237                         invalid number
9238                                    NaN
9239    Will revert after reading the email
Name: Tags, Length: 9240, dtype: object

Let's do One hot encoding for the categorical data columns. first we drop the job position column to reduce feature complexity. and then encode the rest


In [26]:
df = df.drop('Job Position', axis=1)

In [27]:
df.head

<bound method NDFrame.head of          Lead Source Do Not Email Do Not Call  Converted     Country  \
0         Olark Chat           No          No          0       India   
1     Organic Search           No          No          0       India   
2     Direct Traffic           No          No          1       India   
3     Direct Traffic           No          No          0       India   
4             Google           No          No          1       India   
...              ...          ...         ...        ...         ...   
9235  Direct Traffic          Yes          No          1       India   
9236  Direct Traffic           No          No          0       India   
9237  Direct Traffic          Yes          No          0       India   
9238          Google           No          No          1       India   
9239  Direct Traffic           No          No          1  Bangladesh   

                 Specialization Through Recommendations  \
0                        Select               

In [28]:
df['Through Recommendations'].value_counts()

Through Recommendations
No     9233
Yes       7
Name: count, dtype: int64

In [29]:
# encoding binary columns of yes/no
# columns to be encoded:
# 'Through Recommendations'
# 'Do Not Call'
# 'Do Not Email'


encoding_rules = {
    'yes': 1,
    'y': 1,
    'no': 0,
    'n': 0,
}

df['Through Recommendations Encoded'] = df['Through Recommendations'].str.lower().map (encoding_rules).fillna(0)
df['Do Not Call Encoded'] = df['Do Not Call'].str.lower().map (encoding_rules).fillna(0)
df['Do Not Email Encoded'] = df['Do Not Email'].str.lower().map (encoding_rules).fillna(0)

      # All others → 0

In [30]:
df.head(5)

,Lead Source,Do Not Email,Do Not Call,Converted,Country,Specialization,Through Recommendations,Tags,City,Last Notable Activity,Decision_Authority,Through Recommendations Encoded,Do Not Call Encoded,Do Not Email Encoded
0,Olark Chat,No,No,0,India,Select,No,Interested in other courses,Mumbai,Modified,1,0,0,0
1,Organic Search,No,No,0,India,Select,No,Ringing,Mumbai,Email Opened,4,0,0,0
2,Direct Traffic,No,No,1,India,Business Administration,No,Will revert after reading the email,Mumbai,Email Opened,1,0,0,0
3,Direct Traffic,No,No,0,India,Media and Advertising,No,Ringing,Mumbai,Modified,2,0,0,0
4,Google,No,No,1,India,Select,No,Will revert after reading the email,Mumbai,Modified,3,0,0,0


In [31]:
# Drop irrelevant columns
cols_to_drop = ['Through Recommendations','Do Not Call', 'Do Not Email']
df = df.drop(cols_to_drop, axis=1)

In [32]:
df.head(5)

,Lead Source,Converted,Country,Specialization,Tags,City,Last Notable Activity,Decision_Authority,Through Recommendations Encoded,Do Not Call Encoded,Do Not Email Encoded
0,Olark Chat,0,India,Select,Interested in other courses,Mumbai,Modified,1,0,0,0
1,Organic Search,0,India,Select,Ringing,Mumbai,Email Opened,4,0,0,0
2,Direct Traffic,1,India,Business Administration,Will revert after reading the email,Mumbai,Email Opened,1,0,0,0
3,Direct Traffic,0,India,Media and Advertising,Ringing,Mumbai,Modified,2,0,0,0
4,Google,1,India,Select,Will revert after reading the email,Mumbai,Modified,3,0,0,0


let's do some one hot encoding

In [33]:
from sklearn.preprocessing import OneHotEncoder

# Initialize encoder
encoder = OneHotEncoder(sparse_output=False, drop='first')  # Drops first category to avoid multicollinearity

# Fit and transform
encoded_data = encoder.fit_transform(df[['Lead Source', 'Country', 'Specialization', 'Tags', 'City', 'Last Notable Activity']])

# Convert to DataFrame
encoded_df = pd.DataFrame(
    encoded_data,
    columns=encoder.get_feature_names_out(['Lead Source', 'Country', 'Specialization', 'Tags', 'City', 'Last Notable Activity'])
)

# Combine with original data
df_encoded = pd.concat([df.drop(['Lead Source', 'Country', 'Specialization', 'Tags', 'City', 'Last Notable Activity'], axis=1), encoded_df], axis=1)

In [34]:
df_encoded.head(5)

,Converted,Decision_Authority,Through Recommendations Encoded,Do Not Call Encoded,Do Not Email Encoded,Lead Source_Direct Traffic,Lead Source_Facebook,Lead Source_Google,Lead Source_Live Chat,Lead Source_NC_EDM,...,Last Notable Activity_Form Submitted on Website,Last Notable Activity_Had a Phone Conversation,Last Notable Activity_Modified,Last Notable Activity_Olark Chat Conversation,Last Notable Activity_Page Visited on Website,Last Notable Activity_Resubscribed to emails,Last Notable Activity_SMS Sent,Last Notable Activity_Unreachable,Last Notable Activity_Unsubscribed,Last Notable Activity_View in browser link Clicked
0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,4,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,0,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,0,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,3,0,0,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's Split the data

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X = df_encoded.drop('Converted', axis=1)  # All features except target
y = df_encoded['Converted']

X.head(5)
y.head(5)


0    0
1    0
2    1
3    0
4    1
Name: Converted, dtype: int64

In [36]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 20% for testing
    random_state=42,  # Ensures reproducibility
    stratify=y        # Preserves class balance in splits
)# Target variable

Training on Logistic Regression

In [37]:
model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

evaluation

In [38]:
# Predict on test set
y_pred = model.predict(X_test)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9134199134199135
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      1136
           1       0.92      0.85      0.88       712

    accuracy                           0.91      1848
   macro avg       0.92      0.90      0.91      1848
weighted avg       0.91      0.91      0.91      1848



In [39]:
# 3. Get probabilities
y_prob = model.predict_proba(X_test)[:, 1]  # P(conversion)

# 4. Add probabilities back to DataFrame (for analysis)
test_df = X_test.copy()
test_df['conversion_prob'] = y_prob
test_df['actual'] = y_test  # Optional: Compare with ground truth

print(test_df[['conversion_prob', 'actual']].head(20))




      conversion_prob  actual
683          0.097757       1
1931         0.948731       1
6950         0.079564       0
2996         0.005308       0
3902         0.003656       0
6828         0.013781       0
4906         0.779874       0
5796         0.236021       0
6388         0.084496       0
3636         0.160442       0
3685         0.992791       1
3279         0.130908       0
3799         0.271953       0
9158         0.766659       1
5766         0.011137       0
4341         0.951926       1
3043         0.135587       0
1528         0.995351       1
1811         0.263985       0
900          0.894788       1


Trying random forest

In [50]:
print(X_train.head(5))
y_train.head(5)

      Decision_Authority  Through Recommendations Encoded  \
9067                   3                                0   
6093                   3                                0   
855                    2                                0   
6053                   2                                0   
292                    3                                0   

      Do Not Call Encoded  Do Not Email Encoded  Lead Source_Direct Traffic  \
9067                    0                     0                         0.0   
6093                    0                     0                         0.0   
855                     0                     0                         0.0   
6053                    0                     0                         0.0   
292                     0                     0                         1.0   

      Lead Source_Facebook  Lead Source_Google  Lead Source_Live Chat  \
9067                   0.0                 1.0                    0.0   
6093        

9067    1
6093    0
855     1
6053    0
292     0
Name: Converted, dtype: int64

In [40]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train
rf = RandomForestClassifier(n_estimators=100, random_state=42)



rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
# Get raw probabilities (uncalibrated)

Accuracy: 0.9117965367965368
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1136
           1       0.91      0.86      0.88       712

    accuracy                           0.91      1848
   macro avg       0.91      0.90      0.91      1848
weighted avg       0.91      0.91      0.91      1848



In [41]:
y_prob_uncalibrated = rf.predict_proba(X_test)[:, 1]

print(y_prob_uncalibrated)

# 4. Add probabilities back to DataFrame (for analysis)
test_df = X_test.copy()
test_df['conversion_prob'] = y_prob_uncalibrated
test_df['actual'] = y_test  # Optional: Compare with ground truth

print(test_df[['conversion_prob', 'actual']].head(20))

[0.1825     0.68885714 0.48178571 ... 0.48583333 0.02       0.17363564]
      conversion_prob  actual
683          0.182500       1
1931         0.688857       1
6950         0.481786       0
2996         0.010000       0
3902         0.021667       0
6828         0.010000       0
4906         0.945130       0
5796         0.196958       0
6388         0.056624       0
3636         0.039963       0
3685         1.000000       1
3279         0.157691       0
3799         0.311044       0
9158         0.291810       1
5766         0.000000       0
4341         0.960000       1
3043         0.106442       0
1528         1.000000       1
1811         0.238399       0
900          1.000000       1


Trying out both models on synthetic non-linear dataset, RF outperformed LR as we see.

In [42]:

from sklearn.metrics import  roc_auc_score
from sklearn.datasets import make_moons
X_nl, y_nl = make_moons(n_samples=1000, noise=0.3, random_state=42)

# Train both models
lr_nl = LogisticRegression().fit(X_nl, y_nl)
rf_nl = RandomForestClassifier(random_state=42).fit(X_nl, y_nl)

# Compare AUC
print("LR AUC (Non-linear):", roc_auc_score(y_nl, lr_nl.predict_proba(X_nl)[:, 1]))  # Likely poor
print("RF AUC (Non-linear):", roc_auc_score(y_nl, rf_nl.predict_proba(X_nl)[:, 1]))  # Should be >0.9

LR AUC (Non-linear): 0.934744
RF AUC (Non-linear): 1.0


In [43]:
from sklearn.ensemble import GradientBoostingClassifier


# Initialize the Gradient Boosting Regressor
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)


# Fit the model to your training data
gb_model.fit(X_train, y_train)

# Make predictions
y_pred = gb_model.predict(X_test)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))





Accuracy: 0.917
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1136
           1       0.92      0.86      0.89       712

    accuracy                           0.92      1848
   macro avg       0.92      0.91      0.91      1848
weighted avg       0.92      0.92      0.92      1848



In [44]:
# Get predicted probabilities
proba = gb_model.predict_proba(X_test)

# Extract the probability of the positive class (e.g., lead becoming a sale)
lead_sale_prob = proba[:, 1]

# 4. Add probabilities back to DataFrame (for analysis)
test_df = X_test.copy()
test_df['conversion_prob'] = y_prob
test_df['actual'] = y_test  # Optional: Compare with ground truth

print(test_df[['conversion_prob', 'actual']].head(20))



      conversion_prob  actual
683          0.097757       1
1931         0.948731       1
6950         0.079564       0
2996         0.005308       0
3902         0.003656       0
6828         0.013781       0
4906         0.779874       0
5796         0.236021       0
6388         0.084496       0
3636         0.160442       0
3685         0.992791       1
3279         0.130908       0
3799         0.271953       0
9158         0.766659       1
5766         0.011137       0
4341         0.951926       1
3043         0.135587       0
1528         0.995351       1
1811         0.263985       0
900          0.894788       1


Considering that all the models have similar outputs. Random forests seem to be an optimal choice since they capture non-linearity and are less computationally expensive than gradient boosting. Logistic regression yielded good scores but, in the future, if the dataset is more complex, it will probably fail to capture its patterns.

We will further fine-tune and calibrate it.

fine tuning

In [45]:
# # first accuracy:  0.9128
# first f1: 0.88


#  2nd accuracy: 0.9134 , n_estimators=200
#  2nd f1: 0.89

# 3rd accuracy:0.9139  , n_estimators=500
# 3rd f1: 0.89

# according to available litrature, the obtained f1 score is considered good.

from sklearn.ensemble import RandomForestClassifier

# Initialize and train
rf = RandomForestClassifier(n_estimators=500, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
# Get raw probabilities (uncalibrated)

Accuracy: 0.9123376623376623
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1136
           1       0.90      0.86      0.88       712

    accuracy                           0.91      1848
   macro avg       0.91      0.90      0.91      1848
weighted avg       0.91      0.91      0.91      1848



In [46]:
df.head()

,Lead Source,Converted,Country,Specialization,Tags,City,Last Notable Activity,Decision_Authority,Through Recommendations Encoded,Do Not Call Encoded,Do Not Email Encoded
0,Olark Chat,0,India,Select,Interested in other courses,Mumbai,Modified,1,0,0,0
1,Organic Search,0,India,Select,Ringing,Mumbai,Email Opened,4,0,0,0
2,Direct Traffic,1,India,Business Administration,Will revert after reading the email,Mumbai,Email Opened,1,0,0,0
3,Direct Traffic,0,India,Media and Advertising,Ringing,Mumbai,Modified,2,0,0,0
4,Google,1,India,Select,Will revert after reading the email,Mumbai,Modified,3,0,0,0


In [47]:
df = df.rename(columns={'old_column_name': 'new_column_name'})


What we need to do is to rename the columns so they match with the fields names in Odoo itself.
